In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/nfl-big-data-bowl-2020/train.csv
/kaggle/input/nfl-big-data-bowl-2020/kaggle/competitions/nflrush/sample_submission.csv.encrypted
/kaggle/input/nfl-big-data-bowl-2020/kaggle/competitions/nflrush/competition.cpython-36m-x86_64-linux-gnu.so
/kaggle/input/nfl-big-data-bowl-2020/kaggle/competitions/nflrush/test.csv.encrypted
/kaggle/input/nfl-big-data-bowl-2020/kaggle/competitions/nflrush/__init__.py



v1: new 0.75/0.25

v2: 0.8 / 0.2

v4: 0.8/0.2

v6:0.75/0.25

In [2]:
###raw mae
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import catboost as cb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold,TimeSeriesSplit,KFold,GroupKFold
from sklearn.metrics import roc_auc_score,mean_squared_error,mean_absolute_error,log_loss
import sqlite3
import xgboost as xgb
import datetime
from sklearn.linear_model import LogisticRegression
from scipy.stats import pearsonr
import gc
from sklearn.model_selection import TimeSeriesSplit
from bayes_opt import BayesianOptimization
#from kaggle.competitions import nflrush
import re
from string import punctuation
from scipy.spatial import Voronoi
from scipy.spatial import ConvexHull
from scipy.spatial import Delaunay
#env = nflrush.make_env()

In [3]:
pd.options.display.max_columns = 200

In [4]:
train = pd.read_csv('../raw_data/test_stage1//train.csv',low_memory=False)
train.loc[train['Season'] == 2017, 'S'] = (train['S'][train['Season'] == 2017] - 2.4355) / 1.2930 * 1.4551 + 2.7570

In [7]:
train.loc[train.VisitorTeamAbbr == "ARI",'VisitorTeamAbbr'] = "ARZ"
train.loc[train.HomeTeamAbbr == "ARI",'HomeTeamAbbr'] = "ARZ"

train.loc[train.VisitorTeamAbbr == "BAL",'VisitorTeamAbbr'] = "BLT"
train.loc[train.HomeTeamAbbr == "BAL",'HomeTeamAbbr'] = "BLT"

train.loc[train.VisitorTeamAbbr == "CLE",'VisitorTeamAbbr'] = "CLV"
train.loc[train.HomeTeamAbbr == "CLE",'HomeTeamAbbr'] = "CLV"

train.loc[train.VisitorTeamAbbr == "HOU",'VisitorTeamAbbr'] = "HST"
train.loc[train.HomeTeamAbbr == "HOU",'HomeTeamAbbr'] = "HST"

train.loc[train.Season==2017,'Orientation'] = np.mod(train.loc[train.Season==2017,'Orientation'] + 90,360)

E:\software\ANACONDA3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in remainder
  del sys.path[0]


In [8]:
# team_features = list(train.columns).copy()
# person_feature = []
# for i in train.columns:
#     if train.loc[train.PlayId ==20170907000118,i].nunique()!=1 and i!='PlayId':
#             print(i)
#             team_features.remove(i)
#             person_feature.append(i)
person_feature = ['Team',
 'X',
 'Y',
 'S',
 'A',
 'Dis',
 'Orientation',
 'Dir',
 'NflId',
 'DisplayName',
 'JerseyNumber',
 'PlayerHeight',
 'PlayerWeight',
 'PlayerBirthDate',
 'PlayerCollegeName',
 'Position']

In [9]:
train['ToLeft'] = train.PlayDirection == "left"
train['X_std'] = train.X
train.loc[train.ToLeft, 'X_std'] = 120 - train.loc[train.ToLeft, 'X'] 
train['X'] = train['X_std'].copy()
train['Y_std'] = train.Y
train.loc[train.ToLeft, 'Y_std'] = 160/3 - train.loc[train.ToLeft, 'Y'] 
train['Y'] = train['Y_std'].copy()
train['Orientation_std'] = np.mod(train.Orientation-90, 360)
train.loc[train.ToLeft, 'Orientation_std'] = np.mod(180 + train.loc[train.ToLeft, 'Orientation_std'].copy(),360)
train['Orientation'] = train['Orientation_std'].copy()
train['Dir_rad'] = np.mod(train.Dir-90, 360) * np.pi/180.0
train['Dir_std'] = train.Dir_rad.copy()
train.loc[train.ToLeft, 'Dir_std'] = np.mod(np.pi + train.loc[train.ToLeft, 'Dir_rad'], 2*np.pi)
train['Dir'] = train['Dir_std'].copy()
person_feature.append('PlayId')
person_info = train[person_feature].copy()
person_info['player_mark'] = ['player'+ str(i) for i in range(1,12)]*int(person_info.shape[0]/11)
te = pd.pivot_table(person_info,columns=['Team','player_mark'],index=['PlayId'],aggfunc=lambda x:x)
te.columns = ['_'.join(a) for a in te.columns.values]

E:\software\ANACONDA3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in remainder
  
E:\software\ANACONDA3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in remainder
  if __name__ == '__main__':
E:\software\ANACONDA3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in remainder
  # This is added back by InteractiveShellApp.init_path()
E:\software\ANACONDA3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in remainder
  del sys.path[0]


In [10]:
### drop TeamOnoffsen isonoffense
train['TeamOnOffense'] = "home"
train.loc[train.PossessionTeam != train.HomeTeamAbbr, 'TeamOnOffense'] = "away"
train['IsOnOffense'] = train.Team == train.TeamOnOffense

In [11]:
train['is_run'] = train.NflId == train.NflIdRusher
train_single = train[train.is_run==True].copy()
train_single['diff_Ori_dir'] = np.abs(train_single.Orientation - train_single.Dir/np.pi*180)
temp1 = pd.DataFrame(train_single['diff_Ori_dir'].copy())
temp1['c1'] = 360 - temp1.diff_Ori_dir
train_single['diff_Ori_dir']= np.min(temp1,axis=1).tolist()
def transform_time_quarter(str1):
    try:
        return int(str1[:2])*60 + int(str1[3:5])
    except:
        return np.nan
def transform_time_all(str1,quarter):
    try:
        if quarter<=4:
            return 15*60 - (int(str1[:2])*60 + int(str1[3:5])) + (quarter-1)*15*60
        if quarter ==5:
            return 10*60 - (int(str1[:2])*60 + int(str1[3:5])) + (quarter-1)*15*60
    except:
        return np.nan
train_single['time_quarter'] = train_single.GameClock.map(lambda x:transform_time_quarter(x))
train_single['time_end'] = train_single.apply(lambda x:transform_time_all(x.loc['GameClock'],x.loc['Quarter']),axis=1)
train_single['TimeHandoff'] = train_single['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
train_single['TimeSnap'] = train_single['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
train_single['date_game'] = train_single.GameId.map(lambda x:pd.to_datetime(str(x)[:8]))
train_single['runner_age'] = (train_single.date_game.map(pd.to_datetime) - train_single.PlayerBirthDate.map(pd.to_datetime)).map(lambda x:x.days)/365.25
def transform_height(te):
    try:
        if pd.isna(te):
            return np.nan
        return (int(te.split('-')[0])*12 + int(te.split('-')[1]))*2.54/100
    except:
        return np.nan
train_single['runner_height'] = train_single.PlayerHeight.map(transform_height)
train_single['own_field'] = (train_single['FieldPosition'] == train_single['PossessionTeam']).astype(int)
train_single['dist_to_end'] = train_single.apply(lambda x:(100 - x.loc['YardLine']) if x.loc['own_field']==1 else x.loc['YardLine'],axis=1)
def transform_YardLine_x(row):
    try:
        if row.own_field ==1:
            return  row.YardLine + 10
        if row.own_field ==0:
            return 110 - row.YardLine
    except:
        return np.nan
def transform_dist_to_line(row):
    try:
        return row.X - row.YardLine_x
    except:
        return np.nan
train_single['YardLine_x'] = train_single.apply(transform_YardLine_x,axis=1)
train_single['dist_to_line'] = train_single.apply(transform_dist_to_line,axis=1)
train['is_QB'] = train.Position=='QB'
qb_number = train.groupby(['PlayId'])['is_QB'].sum()
train_single['qb_number'] = train_single.PlayId.map(qb_number)
te1 = train.loc[(train.is_run==True)|(train.is_QB==1),['PlayId','X','Y','is_run','is_QB','A','S',
                                                       'Dis','Orientation','Dir','PlayerHeight','PlayerWeight','PlayerBirthDate']]
te1 = te1.set_index(['PlayId'])
run_info = te1[te1.is_run==True].copy()
run_info.rename({'X':'run_x','Y':'run_y'},axis=1,inplace=True)
qb_info = te1[te1.is_QB==1].copy()
qb_info.reset_index(inplace=True)
# 要改万一大于2个
drop_index=[]
for num_index in qb_number.index[qb_number>=2]:
    temp = qb_info.loc[qb_info.PlayId==num_index,['X','Y']]
    dist_to_close = []
    for num in temp.index:
        temp_a = np.sqrt(np.sum((temp.loc[num].values - run_info.loc[run_info.index==num_index,['run_x','run_y']].values[0,])**2))
        #print(num_index)
        dist_to_close.append(temp_a)
    drop_index.extend([temp.index.tolist()[i] for i in list(np.argsort(dist_to_close))[1:]])
qb_info.drop(drop_index,axis=0,inplace=True)
# drop_index=[]
# for num_index in qb_number.index[qb_number==2]:
#     temp = qb_info.loc[qb_info.PlayId==num_index,['X','Y']]
#     dist_to_close = []
#     for num in temp.index:
#         temp_a = np.sqrt(np.sum((temp.loc[num].values - run_info.loc[run_info.index==num_index,['run_x','run_y']].values[0,])**2))
#         #print(num_index)
#         dist_to_close.append(temp_a)
#     if dist_to_close[0]>dist_to_close[1]:
#         drop_index.append(temp.index.tolist()[0])
#     else:
#         drop_index.append(temp.index.tolist()[1])
# qb_info.drop(drop_index,axis=0,inplace=True)
train_single.rename({'X':'run_x','Y':'run_y'},axis=1,inplace=True)
qb_info.rename({'A':'QB_A','S':'QB_S','Dis':'QB_Dis','Orientation':'QB_Orientation',
                'Dir':'QB_dir','PlayerHeight':'QB_PlayerHeight',
                'PlayerWeight':'QB_PlayerWeight','PlayerBirthDate':'QB_PlayerBirthDate'},axis=1,inplace=True)
train_single = pd.merge(train_single,qb_info[['PlayId','X','Y','QB_A','QB_S','QB_Dis','QB_Orientation',
                                             'QB_dir','QB_PlayerHeight','QB_PlayerWeight',
                                             'QB_PlayerBirthDate']],on='PlayId',how='left')
train_single.rename({'X':'QB_x','Y':'QB_y'},axis=1,inplace=True)
def transform_qb_to_line(row):
    try:
        return row.QB_x- row.YardLine_x
    except:
        return np.nan
train_single['qb_to_line'] = train_single.apply(transform_qb_to_line,axis=1)
def transform_run_dist_qb(row):
    try:
        x_dist = row.QB_x - row.run_x
        y_dist = row.QB_y - row.run_y
        return np.sqrt(x_dist**2 + y_dist**2)
    except:
        return np.nan
train_single['run_dist_qb'] = train_single.apply(transform_run_dist_qb,axis=1)
train_single['QB_PlayerHeight'] = train_single.QB_PlayerHeight.map(transform_height)
train_single['QB_age'] = (train_single.date_game.map(pd.to_datetime) - train_single.QB_PlayerBirthDate.map(pd.to_datetime)).map(lambda x:x.days)/365.25

train_single['diff_Ori_dir_qb'] = np.abs(train_single.QB_Orientation - train_single.QB_dir/np.pi*180)
temp1 = pd.DataFrame(train_single['diff_Ori_dir_qb'].copy())
temp1['c1'] = 360 - temp1.diff_Ori_dir_qb
train_single['diff_Ori_dir_qb']= np.min(temp1,axis=1).tolist()

train_single['diff_qb_runner_dir'] = np.abs(train_single.Dir/np.pi*180 - train_single.QB_dir/np.pi*180)
temp1 = pd.DataFrame(train_single['diff_qb_runner_dir'].copy())
temp1['c1'] = 360 - temp1.diff_qb_runner_dir
train_single['diff_qb_runner_dir']= np.min(temp1,axis=1).tolist()

train_single['diff_qb_runner_ori'] = np.abs(train_single.Orientation - train_single.QB_Orientation)
temp1 = pd.DataFrame(train_single['diff_qb_runner_ori'].copy())
temp1['c1'] = 360 - temp1.diff_qb_runner_ori
train_single['diff_qb_runner_ori']= np.min(temp1,axis=1).tolist()

In [12]:

train_single['score_difference'] = train_single.apply(lambda x:
                                    x.loc['HomeScoreBeforePlay'] - x.loc['VisitorScoreBeforePlay'] if x.loc['PossessionTeam'] == x.loc['HomeTeamAbbr']
                                                      else x.loc['VisitorScoreBeforePlay'] - x.loc['HomeScoreBeforePlay'],axis=1)
train_single['offense_score'] = train_single.apply(lambda x:
                                    x.loc['HomeScoreBeforePlay']  if x.loc['PossessionTeam'] == x.loc['HomeTeamAbbr']
                                                      else x.loc['VisitorScoreBeforePlay'] ,axis=1)

In [13]:
remove_features = ['GameId','PlayId','DisplayName','GameClock','TimeHandoff','TimeSnap','TeamOnOffense','IsOnOffense']
remove_features.append('HomeTeamAbbr')
remove_features.append('VisitorTeamAbbr')
remove_features.append('PlayerBirthDate')
remove_features.append('is_run')
remove_features.append('PossessionTeam')
remove_features.append('FieldPosition')
remove_features.append('PlayerHeight')
remove_features.append('NflIdRusher')
remove_features.append('date_game')
remove_features.append('YardLine_x')
remove_features.append('QB_x')
remove_features.append('QB_PlayerBirthDate')
remove_features.extend(te.columns.tolist())
remove_features.append('Turf')
remove_features.extend(['QB_A','QB_Dis','QB_Orientation','QB_PlayerHeight','QB_PlayerWeight','QB_age'])
remove_features.extend(['YardLine','X_std','Y_std','ToLeft','Dir_rad','Dir_std','Orientation_std'])
remove_features.extend(['HomeScoreBeforePlay','VisitorScoreBeforePlay'])

In [14]:
train_single = pd.merge(train_single,te,on='PlayId',how = 'left')

In [15]:
def dist1_divide_S(row):
    try:
        dist_list_def = []
        S_list_def = []
        Dir_list_def = []
        X_list_def  = []
        if row.Team == 'home':
            team_def = 'away'
        else:
            team_def = 'home'
        for i in range(1,12):
            x_str = 'X_'+team_def+'_player'+str(i)
            y_str = 'Y_'+team_def+'_player'+str(i)
            if row[x_str] < row.run_x:
                continue
            dist1 = np.sqrt((row[x_str] - row.run_x)**2 + (row[y_str] - row.run_y)**2)
            dist_list_def.append(dist1)
            X_list_def.append(row['X_'+team_def+'_player'+str(i)])
            Dir_list_def.append(row['Dir_'+team_def+'_player'+str(i)])
            S_list_def.append(row['S_'+team_def+'_player'+str(i)])
            S_max = np.array(S_list_def)[np.argsort(dist_list_def)][0]
            Dir_max = np.array(Dir_list_def)[np.argsort(dist_list_def)][0]
            X_max = np.array(X_list_def)[np.argsort(dist_list_def)][0]
            S_hor_def  = np.abs(S_max * np.cos(Dir_max))
        if sorted(dist_list_def)[0]/S_max == np.inf:
            return 999
        return np.clip(sorted(dist_list_def)[0]/S_max,0,999)
    except:
        return np.nan
train_single['dist1_divide_S'] = train_single.apply(dist1_divide_S,axis=1)

E:\software\ANACONDA3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in double_scalars


In [16]:
def dist1_divide_S_hor(row):
    try:
        dist_list_def = []
        S_list_def = []
        Dir_list_def = []
        X_list_def  = []
        if row.Team == 'home':
            team_def = 'away'
        else:
            team_def = 'home'
        for i in range(1,12):
            x_str = 'X_'+team_def+'_player'+str(i)
            y_str = 'Y_'+team_def+'_player'+str(i)
            if row[x_str] < row.run_x:
                continue
            dist1 = np.sqrt((row[x_str] - row.run_x)**2 + (row[y_str] - row.run_y)**2)
            dist_list_def.append(dist1)
            X_list_def.append(row['X_'+team_def+'_player'+str(i)])
            Dir_list_def.append(row['Dir_'+team_def+'_player'+str(i)])
            S_list_def.append(row['S_'+team_def+'_player'+str(i)])
            S_max = np.array(S_list_def)[np.argsort(dist_list_def)][0]
            Dir_max = np.array(Dir_list_def)[np.argsort(dist_list_def)][0]
            X_max = np.array(X_list_def)[np.argsort(dist_list_def)][0]
            S_hor_def  = np.abs(S_max * np.cos(Dir_max))
            temp = (X_max - row.run_x)/S_hor_def
        if temp == np.inf:
            return 999
        return np.clip(temp,0,999)
    except:
        return np.nan
train_single['dist1_divide_S_hor'] = train_single.apply(dist1_divide_S_hor,axis=1)

E:\software\ANACONDA3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in double_scalars


In [17]:
def time1_dist_S(row):
    try:
        dist_list_def = []
        S_list_def = []
        Dir_list_def = []
        X_list_def  = []
        time_list_def = []
        if row.Team == 'home':
            team_def = 'away'
        else:
            team_def = 'home'
        for i in range(1,12):
            x_str = 'X_'+team_def+'_player'+str(i)
            y_str = 'Y_'+team_def+'_player'+str(i)
            if row[x_str] < row.run_x:
                continue
            dist1 = np.sqrt((row[x_str] - row.run_x)**2 + (row[y_str] - row.run_y)**2)
            dist_list_def.append(dist1)
            time_list_def.append(dist1/row['S_'+team_def+'_player'+str(i)])
        return np.clip(sorted(time_list_def)[0],0,999)
    except:
        return np.nan
train_single['time1_dist_S'] = train_single.apply(time1_dist_S,axis=1)

E:\software\ANACONDA3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in double_scalars


In [18]:
def time1_dist_S_hor(row):
    try:
        dist_list_def = []
        S_list_def = []
        Dir_list_def = []
        X_list_def  = []
        time_list_def = []
        if row.Team == 'home':
            team_def = 'away'
        else:
            team_def = 'home'
        for i in range(1,12):
            x_str = 'X_'+team_def+'_player'+str(i)
            y_str = 'Y_'+team_def+'_player'+str(i)
            if row[x_str] < row.run_x:
                continue
            dist1 = np.sqrt((row[x_str] - row.run_x)**2 + (row[y_str] - row.run_y)**2)
            time_list_def.append((row[x_str] - row.run_x)/np.abs(row['S_'+team_def+'_player'+str(i)] * np.cos(row['Dir_'+team_def+'_player'+str(i)])))
        return np.clip(sorted(time_list_def)[0],0,999)
    except:
        return np.nan
train_single['time1_dist_S_hor'] = train_single.apply(time1_dist_S_hor,axis=1)

E:\software\ANACONDA3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in double_scalars


In [19]:
def transform_close_who(row):
    try:
        dist_list_off = []
        if row.Team == 'home':
            team_off = 'home'
        else:
            team_off = 'away'
        for i in range(1,12):
            if row['Position_'+team_off+'_player'+str(i)] =='QB':
                continue
            if row['X_'+team_off+'_player'+str(i)]<row.run_x:
                continue
            x_str = 'X_'+team_off+'_player'+str(i)
            y_str = 'Y_'+team_off+'_player'+str(i)
            dist1 = np.sqrt((row[x_str] - row.run_x)**2 + (row[y_str] - row.run_y)**2)
            dist_list_off.append(dist1)

        dist_list_def = []
        S_list_def = []
        if row.Team == 'home':
            team_def = 'away'
        else:
            team_def = 'home'
        for i in range(1,12):
            x_str = 'X_'+team_def+'_player'+str(i)
            y_str = 'Y_'+team_def+'_player'+str(i)
            if row[x_str] < row.run_x:
                continue
            dist1 = np.sqrt((row[x_str] - row.run_x)**2 + (row[y_str] - row.run_y)**2)
            dist_list_def.append(dist1)
            S_list_def.append(row['S_'+team_def+'_player'+str(i)])
        dist_list_off = list(filter(lambda x: x!=0,dist_list_off))
    #    return dist_list_def,list(filter(lambda x: x!=0,dist_list_off))
        if len(dist_list_def) ==0 or len(dist_list_off)==0:
            return np.nan
        if np.min(dist_list_def) <= np.min(dist_list_off):
            return (np.array(S_list_def)[np.argsort(dist_list_def)][0])/row.S
        else:
            return 0
    except:
        return np.nan

In [20]:
train_single['close_who_S_ratio']= train_single.apply(transform_close_who,axis=1)

In [21]:
def transform_dist_close123_defender_runner(row):
    try:
        ###return the attributes of 3 closest defender
        dist_list = []
        if row.Team == 'home':
            team_def = 'away'
        else:
            team_def = 'home'
        for i in range(1,12):
            x_str = 'X_'+team_def+'_player'+str(i)
            y_str = 'Y_'+team_def+'_player'+str(i)
            dist1 = np.sqrt((row[x_str] - row.run_x)**2 + (row[y_str] - row.run_y)**2)
            dist_list.append(dist1)
        first_close = sorted(dist_list)[0]
        min1_num = dist_list.index(first_close)+1
        A_min1_dist = row['A_'+team_def+'_player'+str(min1_num)]
        S_min1_dist = row['S_'+team_def+'_player'+str(min1_num)]

        first2_close = sorted(dist_list)[1]
        min2_num = dist_list.index(first2_close)+1
        A_min2_dist = row['A_'+team_def+'_player'+str(min2_num)]
        S_min2_dist = row['S_'+team_def+'_player'+str(min2_num)]

        first3_close = sorted(dist_list)[2]
        min3_num = dist_list.index(first3_close)+1
        A_min3_dist = row['A_'+team_def+'_player'+str(min3_num)]
        S_min3_dist = row['S_'+team_def+'_player'+str(min3_num)]

        first4_close = sorted(dist_list)[3]
        min4_num = dist_list.index(first4_close)+1
        A_min4_dist = row['A_'+team_def+'_player'+str(min4_num)]
        S_min4_dist = row['S_'+team_def+'_player'+str(min4_num)]

        first5_close = sorted(dist_list)[4]
        min5_num = dist_list.index(first5_close)+1
        A_min5_dist = row['A_'+team_def+'_player'+str(min5_num)]
        S_min5_dist = row['S_'+team_def+'_player'+str(min5_num)]

        return first_close,A_min1_dist,S_min1_dist,first2_close,A_min2_dist,S_min2_dist,first3_close,A_min3_dist,S_min3_dist,first4_close,A_min4_dist,S_min4_dist,first5_close,A_min5_dist,S_min5_dist
    except:
        return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
def transform_dist_close123_offense_runner(row):
    try:
        dist_list = []
        if row.Team == 'home':
            team_def = 'home'
        else:
            team_def = 'away'
        for i in range(1,12):
            x_str = 'X_'+team_def+'_player'+str(i)
            y_str = 'Y_'+team_def+'_player'+str(i)
            dist1 = np.sqrt((row[x_str] - row.run_x)**2 + (row[y_str] - row.run_y)**2)
            dist_list.append(dist1)
        first_close = sorted(dist_list)[1]
        min1_num = dist_list.index(first_close)+1
        A_min1_dist = row['A_'+team_def+'_player'+str(min1_num)]
        S_min1_dist = row['S_'+team_def+'_player'+str(min1_num)]

        first2_close = sorted(dist_list)[2]
        min2_num = dist_list.index(first2_close)+1
        A_min2_dist = row['A_'+team_def+'_player'+str(min2_num)]
        S_min2_dist = row['S_'+team_def+'_player'+str(min2_num)]

        first3_close = sorted(dist_list)[3]
        min3_num = dist_list.index(first3_close)+1
        A_min3_dist = row['A_'+team_def+'_player'+str(min3_num)]
        S_min3_dist = row['S_'+team_def+'_player'+str(min3_num)]

        first4_close = sorted(dist_list)[4]
        min4_num = dist_list.index(first4_close)+1
        A_min4_dist = row['A_'+team_def+'_player'+str(min4_num)]
        S_min4_dist = row['S_'+team_def+'_player'+str(min4_num)]

        first5_close = sorted(dist_list)[5]
        min5_num = dist_list.index(first5_close)+1
        A_min5_dist = row['A_'+team_def+'_player'+str(min5_num)]
        S_min5_dist = row['S_'+team_def+'_player'+str(min5_num)]

        return first_close,A_min1_dist,S_min1_dist,first2_close,A_min2_dist,S_min2_dist,first3_close,A_min3_dist,S_min3_dist,first4_close,A_min4_dist,S_min4_dist,first5_close,A_min5_dist,S_min5_dist
    except:
        return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan

In [22]:
temp1  = train_single.apply(transform_dist_close123_offense_runner,axis=1)
names = ['dist_1_off','A_1_off','S_1_off','dist_2_off','A_2_off','S_2_off','dist_3_off','A_3_off','S_3_off',
        'dist_4_off','A_4_off','S_4_off','dist_5_off','A_5_off','S_5_off']
for i in range(len(names)):
    train_single[names[i]] = temp1.map(lambda x:x[i])

temp1  = train_single.apply(transform_dist_close123_defender_runner,axis=1)
names = ['dist_1_def','A_1_def','S_1_def','dist_2_def','A_2_def','S_2_def','dist_3_def','A_3_def','S_3_def',
        'dist_4_def','A_4_def','S_4_def','dist_5_def','A_5_def','S_5_def']
for i in range(len(names)):
    train_single[names[i]] = temp1.map(lambda x:x[i])
for i in [1]:
    train_single['dist_'+str(i)+'_S_ratio'] = train_single['S_'+str(i)+'_def'] / train_single.S
    train_single['dist_'+str(i)+'_S_ratio'] = np.where(np.isinf(train_single['dist_'+str(i)+'_S_ratio']),999,train_single['dist_'+str(i)+'_S_ratio'])

In [23]:
def transform_dist_close123_defender_QB(row):
    try:
        if pd.isna(row.QB_x):
            return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
        dist_list = []
        if row.Team == 'home':
            team_def = 'away'
        else:
            team_def = 'home'
        for i in range(1,12):
            x_str = 'X_'+team_def+'_player'+str(i)
            y_str = 'Y_'+team_def+'_player'+str(i)
            dist1 = np.sqrt((row[x_str] - row.QB_x)**2 + (row[y_str] - row.QB_y)**2)
            dist_list.append(dist1)
        first_close = sorted(dist_list)[0]
        min1_num = dist_list.index(first_close)+1
        A_min1_dist = row['A_'+team_def+'_player'+str(min1_num)]
        S_min1_dist = row['S_'+team_def+'_player'+str(min1_num)]

        first2_close = sorted(dist_list)[1]
        min2_num = dist_list.index(first2_close)+1
        A_min2_dist = row['A_'+team_def+'_player'+str(min2_num)]
        S_min2_dist = row['S_'+team_def+'_player'+str(min2_num)]

        first3_close = sorted(dist_list)[2]
        min3_num = dist_list.index(first3_close)+1
        A_min3_dist = row['A_'+team_def+'_player'+str(min3_num)]
        S_min3_dist = row['S_'+team_def+'_player'+str(min3_num)]

        first4_close = sorted(dist_list)[3]
        min4_num = dist_list.index(first4_close)+1
        A_min4_dist = row['A_'+team_def+'_player'+str(min4_num)]
        S_min4_dist = row['S_'+team_def+'_player'+str(min4_num)]

        first5_close = sorted(dist_list)[4]
        min5_num = dist_list.index(first5_close)+1
        A_min5_dist = row['A_'+team_def+'_player'+str(min5_num)]
        S_min5_dist = row['S_'+team_def+'_player'+str(min5_num)]

        return first_close,A_min1_dist,S_min1_dist,first2_close,A_min2_dist,S_min2_dist,first3_close,A_min3_dist,S_min3_dist,first4_close,A_min4_dist,S_min4_dist,first5_close,A_min5_dist,S_min5_dist
    except:
        return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan

def transform_dist_close123_offense_QB(row):
    try:
        if pd.isna(row.QB_x):
            return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
        dist_list = []
        if row.Team == 'home':
            team_def = 'home'
        else:
            team_def = 'away'
        for i in range(1,12):
            x_str = 'X_'+team_def+'_player'+str(i)
            y_str = 'Y_'+team_def+'_player'+str(i)
            dist1 = np.sqrt((row[x_str] - row.QB_x)**2 + (row[y_str] - row.QB_y)**2)
            dist_list.append(dist1)
        first_close = sorted(dist_list)[1]
        min1_num = dist_list.index(first_close)+1
        A_min1_dist = row['A_'+team_def+'_player'+str(min1_num)]
        S_min1_dist = row['S_'+team_def+'_player'+str(min1_num)]

        first2_close = sorted(dist_list)[2]
        min2_num = dist_list.index(first2_close)+1
        A_min2_dist = row['A_'+team_def+'_player'+str(min2_num)]
        S_min2_dist = row['S_'+team_def+'_player'+str(min2_num)]

        first3_close = sorted(dist_list)[3]
        min3_num = dist_list.index(first3_close)+1
        A_min3_dist = row['A_'+team_def+'_player'+str(min3_num)]
        S_min3_dist = row['S_'+team_def+'_player'+str(min3_num)]

        first4_close = sorted(dist_list)[4]
        min4_num = dist_list.index(first4_close)+1
        A_min4_dist = row['A_'+team_def+'_player'+str(min4_num)]
        S_min4_dist = row['S_'+team_def+'_player'+str(min4_num)]

        first5_close = sorted(dist_list)[5]
        min5_num = dist_list.index(first5_close)+1
        A_min5_dist = row['A_'+team_def+'_player'+str(min5_num)]
        S_min5_dist = row['S_'+team_def+'_player'+str(min5_num)]
        return first_close,A_min1_dist,S_min1_dist,first2_close,A_min2_dist,S_min2_dist,first3_close,A_min3_dist,S_min3_dist,first4_close,A_min4_dist,S_min4_dist,first5_close,A_min5_dist,S_min5_dist
    except:
        return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan

In [24]:
temp1  = train_single.apply(transform_dist_close123_offense_QB,axis=1)
names = ['dist_1_off_qb','A_1_off_qb','S_1_off_qb','dist_2_off_qb','A_2_off_qb','S_2_off_qb','dist_3_off_qb','A_3_off_qb','S_3_off_qb',
        'dist_4_off_qb','A_4_off_qb','S_4_off_qb','dist_5_off_qb','A_5_off_qb','S_5_off_qb']
for i in range(len(names)):
    train_single[names[i]] = temp1.map(lambda x:x[i])

temp1  = train_single.apply(transform_dist_close123_defender_QB,axis=1)
names = ['dist_1_def_qb','A_1_def_qb','S_1_def_qb','dist_2_def_qb','A_2_def_qb','S_2_def_qb','dist_3_def_qb','A_3_def_qb','S_3_def_qb',
        'dist_4_def_qb','A_4_def_qb','S_4_def_qb','dist_5_def_qb','A_5_def_qb','S_5_def_qb']
for i in range(len(names)):
    train_single[names[i]] = temp1.map(lambda x:x[i])

In [25]:
def transform_dist2_num_defender(row):
    try:
        #### 5/10
        dist_dict = {}
        if row.Team == 'home':
            team_def = 'away'
        else:
            team_def = 'home'
        for i in range(1,12):
            x_str = 'X_'+team_def+'_player'+str(i)
            y_str = 'Y_'+team_def+'_player'+str(i)
            dist1 = np.sqrt((row[x_str] - row.run_x)**2 + (row[y_str] - row.run_y)**2)
            dist_dict[i] = dist1

        want_list5 = []
        for a,b in dist_dict.items():
            if b<=5:
                want_list5.append(a)
        num5 = len(want_list5)
        speed_list5 = []
        A_list5 = []
        for i in want_list5:
            speed_list5.append(row['S_'+team_def+'_player'+str(i)])
            A_list5.append(row['A_'+team_def+'_player'+str(i)])
        want_list10 = []
        for a,b in dist_dict.items():
            if b<=10:
                want_list10.append(a)
        num10 = len(want_list10)
        speed_list10 = []
        A_list10 = []
        for i in want_list10:
            speed_list10.append(row['S_'+team_def+'_player'+str(i)])
            A_list10.append(row['A_'+team_def+'_player'+str(i)])
        return num5,np.mean(speed_list5),np.mean(A_list5),num10,np.mean(speed_list10),np.mean(A_list10)
    except:
        return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
    
def transform_dist2_num_offense(row):
    try:
        dist_dict = {}
        if row.Team == 'home':
            team_def = 'home'
        else:
            team_def = 'away'
        for i in range(1,12):
            x_str = 'X_'+team_def+'_player'+str(i)
            y_str = 'Y_'+team_def+'_player'+str(i)
            dist1 = np.sqrt((row[x_str] - row.run_x)**2 + (row[y_str] - row.run_y)**2)
            dist_dict[i] = dist1

        want_list5 = []
        for a,b in dist_dict.items():
            if b<=2:
                want_list5.append(a)
        num5 = len(want_list5)
        speed_list5 = []
        A_list5 = []
        for i in want_list5:
            speed_list5.append(row['S_'+team_def+'_player'+str(i)])
            A_list5.append(row['A_'+team_def+'_player'+str(i)])
        want_list10 = []
        for a,b in dist_dict.items():
            if b<=5:
                want_list10.append(a)
        num10 = len(want_list10)
        speed_list10 = []
        A_list10 = []
        for i in want_list10:
            speed_list10.append(row['S_'+team_def+'_player'+str(i)])
            A_list10.append(row['A_'+team_def+'_player'+str(i)])
        want_list15 = []
        for a,b in dist_dict.items():
            if b<=10:
                want_list15.append(a)
        num15 = len(want_list15)
        speed_list15 = []
        A_list15 = []
        for i in want_list15:
            speed_list15.append(row['S_'+team_def+'_player'+str(i)])
            A_list15.append(row['A_'+team_def+'_player'+str(i)])
        return num5,np.mean(speed_list5),np.mean(A_list5),num10,np.mean(speed_list10),np.mean(A_list10),num15,np.mean(speed_list15),np.mean(A_list15)
    except:
        return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan


In [26]:
temp1  = train_single.apply(transform_dist2_num_defender,axis=1)
names = ['num_dist5_def','S_5_def_mean','A_5_def_mean','num_dist10_def','S_10_def_mean','A_10_def_mean']
for i in range(len(names)):
    train_single[names[i]] = temp1.map(lambda x:x[i])

temp1  = train_single.apply(transform_dist2_num_offense,axis=1)
names = ['num_dist2_off','S_2_off_mean','A_2_off_mean','num_dist5_off','S_5_off_mean','A_5_off_mean','num_dist10_off','S_10_off_mean','A_10_off_mean']
for i in range(len(names)):
    train_single[names[i]] = temp1.map(lambda x:x[i])

E:\software\ANACONDA3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
E:\software\ANACONDA3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [27]:
def transform_dist2_num_defender_QB(row):
    try:
        if pd.isna(row.QB_x):
            return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
        #### 5/10
        dist_dict = {}
        if row.Team == 'home':
            team_def = 'away'
        else:
            team_def = 'home'
        for i in range(1,12):
            x_str = 'X_'+team_def+'_player'+str(i)
            y_str = 'Y_'+team_def+'_player'+str(i)
            dist1 = np.sqrt((row[x_str] - row.QB_x)**2 + (row[y_str] - row.QB_y)**2)
            dist_dict[i] = dist1

        want_list5 = []
        for a,b in dist_dict.items():
            if b<=5:
                want_list5.append(a)
        num5 = len(want_list5)
        speed_list5 = []
        A_list5 = []
        for i in want_list5:
            speed_list5.append(row['S_'+team_def+'_player'+str(i)])
            A_list5.append(row['A_'+team_def+'_player'+str(i)])
        want_list10 = []
        for a,b in dist_dict.items():
            if b<=10:
                want_list10.append(a)
        num10 = len(want_list10)
        speed_list10 = []
        A_list10 = []
        for i in want_list10:
            speed_list10.append(row['S_'+team_def+'_player'+str(i)])
            A_list10.append(row['A_'+team_def+'_player'+str(i)])
        return num5,np.mean(speed_list5),np.mean(A_list5),num10,np.mean(speed_list10),np.mean(A_list10)
    except:
        return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
        
    
def transform_dist2_num_offense_QB(row):
    try:
        if pd.isna(row.QB_x):
            return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
        ### 2/5
        dist_dict = {}
        if row.Team == 'home':
            team_def = 'home'
        else:
            team_def = 'away'
        for i in range(1,12):
            x_str = 'X_'+team_def+'_player'+str(i)
            y_str = 'Y_'+team_def+'_player'+str(i)
            dist1 = np.sqrt((row[x_str] - row.QB_x)**2 + (row[y_str] - row.QB_y)**2)
            dist_dict[i] = dist1

        want_list5 = []
        for a,b in dist_dict.items():
            if b<=2:
                want_list5.append(a)
        num5 = len(want_list5)
        speed_list5 = []
        A_list5 = []
        for i in want_list5:
            speed_list5.append(row['S_'+team_def+'_player'+str(i)])
            A_list5.append(row['A_'+team_def+'_player'+str(i)])
        want_list10 = []
        for a,b in dist_dict.items():
            if b<=5:
                want_list10.append(a)
        num10 = len(want_list10)
        speed_list10 = []
        A_list10 = []
        for i in want_list10:
            speed_list10.append(row['S_'+team_def+'_player'+str(i)])
            A_list10.append(row['A_'+team_def+'_player'+str(i)])
        want_list15 = []
        for a,b in dist_dict.items():
            if b<=10:
                want_list15.append(a)
        num15 = len(want_list15)
        speed_list15 = []
        A_list15 = []
        for i in want_list15:
            speed_list15.append(row['S_'+team_def+'_player'+str(i)])
            A_list15.append(row['A_'+team_def+'_player'+str(i)])
        return num5,np.mean(speed_list5),np.mean(A_list5),num10,np.mean(speed_list10),np.mean(A_list10),num15,np.mean(speed_list15),np.mean(A_list15)
    except:
        return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan


In [28]:
temp1  = train_single.apply(transform_dist2_num_defender_QB,axis=1)
names = ['num_dist5_def_QB','S_5_def_QB_mean','A_5_def_QB_mean','num_dist10_def_QB','S_10_def_QB_mean','A_10_def_QB_mean']
for i in range(len(names)):
    train_single[names[i]] = temp1.map(lambda x:x[i])

temp1  = train_single.apply(transform_dist2_num_offense_QB,axis=1)
names = ['num_dist2_off_QB','S_2_off_QB_mean','A_2_off_QB_mean','num_dist5_off_QB','S_5_off_QB_mean','A_5_off_QB_mean','num_dist10_off_QB','S_10_off_QB_mean','A_10_off_QB_mean']
for i in range(len(names)):
    train_single[names[i]] = temp1.map(lambda x:x[i])

In [29]:
### std of offense defense and all on x y
def transform_defend_X_Y_std(row):
    try:
        def_X = []
        def_Y = []
        if row.Team == 'home':
            team_def = 'away'
        else:
            team_def = 'home'
        for i in range(1,12):
            x_str = 'X_'+team_def+'_player'+str(i)
            y_str = 'Y_'+team_def+'_player'+str(i)
            def_X.append(row[x_str])
            def_Y.append(row[y_str])
        return np.std(def_X),np.std(def_Y)
    except:
        return np.nan,np.nan

def transform_offense_X_Y_std(row):
    try:
        def_X = []
        def_Y = []
        if row.Team == 'home':
            team_def = 'home'
        else:
            team_def = 'away'
        for i in range(1,12):
            x_str = 'X_'+team_def+'_player'+str(i)
            y_str = 'Y_'+team_def+'_player'+str(i)
            def_X.append(row[x_str])
            def_Y.append(row[y_str])
        return np.std(def_X),np.std(def_Y)
    except:
        return np.nan, np.nan

def transform_all_X_Y_std(row):
    try:
        def_X = []
        def_Y = []
        if row.Team == 'home':
            team_def = 'home'
        else:
            team_def = 'away'
        for i in range(1,12):
            x_str = 'X_'+team_def+'_player'+str(i)
            y_str = 'Y_'+team_def+'_player'+str(i)
            def_X.append(row[x_str])
            def_Y.append(row[y_str])

        if row.Team == 'home':
            team_def = 'away'
        else:
            team_def = 'home'
        for i in range(1,12):
            x_str = 'X_'+team_def+'_player'+str(i)
            y_str = 'Y_'+team_def+'_player'+str(i)
            def_X.append(row[x_str])
            def_Y.append(row[y_str])
        return np.std(def_X),np.std(def_Y)
    except:
        return np.nan, np.nan

In [30]:
temp1  = train_single.apply(transform_defend_X_Y_std,axis=1)
names = ['std_def_X','std_def_Y']
for i in range(len(names)):
    train_single[names[i]] = temp1.map(lambda x:x[i])

temp1  = train_single.apply(transform_offense_X_Y_std,axis=1)
names = ['std_off_X','std_off_Y']
for i in range(len(names)):
    train_single[names[i]] = temp1.map(lambda x:x[i])
    
temp1  = train_single.apply(transform_all_X_Y_std,axis=1)
names = ['std_all_X','std_all_Y']
for i in range(len(names)):
    train_single[names[i]] = temp1.map(lambda x:x[i])

In [31]:
def OffensePersonnelSplit(x):
    dic = {'DB' : 0, 'DL' : 0, 'LB' : 0, 'OL' : 0, 'QB' : 0, 'RB' : 0, 'TE' : 0, 'WR' : 0}
    for xx in x.split(","):
        xxs = xx.split(" ")
        if xxs[-1] in dic:
            dic[xxs[-1]] = int(xxs[-2])
    return dic

def DefensePersonnelSplit(x):
    dic = {'DB' : 0, 'DL' : 0, 'LB' : 0, 'OL' : 0}
    for xx in x.split(","):
        xxs = xx.split(" ")
        if xxs[-1] in dic:
            dic[xxs[-1]] = int(xxs[-2])
    return dic
temp1 = train_single.OffensePersonnel.apply(lambda x: pd.Series(OffensePersonnelSplit(x)))
names = ['DB','DL','LB','OL','QB','RB','TE','WR']
for i in range(len(names)):
    train_single['off_num_' + names[i]] = temp1.apply(lambda x:x[names[i]],axis=1)
    
temp1 = train_single.DefensePersonnel.apply(lambda x: pd.Series(DefensePersonnelSplit(x)))
names = ['DB','DL','LB','OL']
for i in range(len(names)):
    train_single['Def_num_' + names[i]] = temp1.apply(lambda x:x[names[i]],axis=1)

In [32]:
train_single['S_horizontal'] = np.cos(train_single.Dir) * train_single.S
train_single['S_vertical'] = np.abs(np.sin(train_single.Dir) * train_single.S)


In [33]:
def is_in_poly(polygon,array):
    try:
        k = 0
        temp = Delaunay(polygon)
        for i in range(array.shape[0]):
            if temp.find_simplex(array[i,:]) >=0:
                k+=1
        return k
    except:
        return 0

def calculate_voroni(row):
    try:
        df=train[train.PlayId==row.PlayId]
        df_offense = df[(df.NflIdRusher != df.NflId)&(df.IsOnOffense)].copy()
        pp = df_offense[['X_std','Y_std']].values.copy()
        df = df[(df.NflIdRusher == df.NflId) | ~df.IsOnOffense] 
        xy = df[['X_std', 'Y_std']].values
        n_points = xy.shape[0]
        xy1 = xy.copy()
        xy1[:,1] = - xy[:,1]
        xy2 = xy.copy()
        xy2[:,1] = 320/3 - xy[:,1]
        xy3 = xy.copy()
        xy3[:,0] =  - xy[:,0]
        xy4 = xy.copy()
        xy4[:,0] = 240 - xy[:,0]
        xy = np.concatenate((xy, xy1, xy2, xy3, xy4), axis=0)
        offense = df.IsOnOffense.values
        vor = Voronoi(xy)
        num_adj = []
        adj_vertice = []
        X_temp = []
        Y_temp = []
        S_temp = []
        Dir_temp = []
        for r in range(n_points):
            if offense[r]:
                rusher_num = r
                rusher_region_points = vor.regions[vor.point_region[r]]
        for r in range(n_points):
            region = vor.regions[vor.point_region[r]]
            if not -1 in region:
                polygon = [vor.vertices[i] for i in region]
                if offense[r]:
                    temp_area = ConvexHull(polygon)
                    voroni_area = temp_area.volume
                    x_max = np.max(np.array(polygon)[:,0])
                    x_min = np.min(np.array(polygon)[:,0])
                else:
                    if len(set(rusher_region_points).intersection(set(region)))>=1:
                        num_adj.append(r)
                        adj_vertice.append(polygon)
        X_temp = df.X_std.values.copy()[num_adj]
        X_Y_temp = df[['X_std','Y_std']].values.copy()[num_adj,:]
        S_temp = df.S.values.copy()[num_adj]
        Dir_temp = df.Dir.values.copy()[num_adj]
        num_in_adj_cell = [is_in_poly(np.array(poly),pp) for poly in adj_vertice]
        num_0_cell = len(list(filter(lambda x:x==0,num_in_adj_cell)))
        sum_num_adj_cell = np.sum(num_in_adj_cell)

        if num_0_cell==0:
            dist1 = np.nan
            S_ratio = np.nan
            time1 = np.nan
            time1_h = np.nan
        else:
            #dist1 = np.min(np.sqrt(np.sum((X_Y_temp[list(np.array(num_in_adj_cell)==0),:] - np.array([row.run_x,row.run_y]))**2,axis=1)))
            S_ratio = np.clip(np.max(S_temp[list(np.array(num_in_adj_cell)==0)]/row.S),0,300)
            #time1 = np.clip(np.min(np.sqrt(np.sum((X_Y_temp[list(np.array(num_in_adj_cell)==0),:] - np.array([row.run_x,row.run_y]))**2,axis=1))/S_temp[list(np.array(num_in_adj_cell)==0)]),0,500)
            time1_h = np.clip(np.min(np.abs(X_temp[list(np.array(num_in_adj_cell)==0)] - row.run_x)/np.abs(S_temp[list(np.array(num_in_adj_cell)==0)]*np.cos(Dir_temp[list(np.array(num_in_adj_cell)==0)]))),0,500)
        return voroni_area,110- x_max,x_max-row.run_x, x_max - row.YardLine_x,len(adj_vertice),sum_num_adj_cell,num_0_cell,S_ratio,time1_h
    except:
        return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan


te = train_single.apply(calculate_voroni,axis=1)
names = ['voroni_area','voroni_bc_close','voroni_bc_x_max_minus_x','voroni_max_yardline','adj_cell_num','adj_off_num','adj_def_num_no_offense','voro0_S_ratio_max','voro0_time_min_h']
for i in range(len(names)):
    train_single[names[i]] = te.map(lambda x:x[i])
train_single['voro_diff_num'] = train_single['adj_cell_num'] - train_single['adj_off_num']

E:\software\ANACONDA3\lib\site-packages\ipykernel_launcher.py:69: RuntimeWarning: divide by zero encountered in true_divide
E:\software\ANACONDA3\lib\site-packages\ipykernel_launcher.py:71: RuntimeWarning: divide by zero encountered in true_divide


In [34]:
###注意下在test的情况
train_single['mean_player_100'] = 0
train_single['std_player_100'] = 0
train_single['min_player_100'] = 0
train_single['quan25_player_100'] = 0
train_single['quan75_player_100'] = 0
for player in list(train_single.NflId.unique()):

    temp = train_single.loc[train_single.NflId==player,'Yards']
    mean1 = temp.shift(1).rolling(100,min_periods=20).mean()
    std1 = temp.shift(1).rolling(100,min_periods=20).std()
    min1 = temp.shift(1).rolling(100,min_periods=20).min()
    quan25 = temp.shift(1).rolling(100,min_periods=20).quantile(0.10)
    quan75 = temp.shift(1).rolling(100,min_periods=20).quantile(0.90)
    train_single.loc[mean1.index,'mean_player_100'] = mean1
    train_single.loc[mean1.index,'std_player_100'] = std1
    train_single.loc[mean1.index,'min_player_100'] = min1
    train_single.loc[mean1.index,'quan25_player_100'] = quan25
    train_single.loc[mean1.index,'quan75_player_100'] = quan75
    
    


In [35]:

train_single['DefendersInTheBox_vs_Distance'] = train_single['DefendersInTheBox'] / train_single['Distance']


In [36]:
def clean_WindDirection(txt):
    try:
        if pd.isna(txt):
            return np.nan
        txt = txt.lower()
        txt = ''.join([c for c in txt if c not in punctuation])
        txt = txt.replace('from', '')
        txt = txt.replace(' ', '')
        txt = txt.replace('north', 'n')
        txt = txt.replace('south', 's')
        txt = txt.replace('west', 'w')
        txt = txt.replace('east', 'e')
        return txt
    except:
        return np.nan
train_single['WindDirection'] = train_single['WindDirection'].apply(clean_WindDirection)
def transform_WindDirection(txt):
    try:
        if pd.isna(txt):
            return np.nan

        if txt=='n':
            return 0
        if txt=='nne' or txt=='nen':
            return 1/8
        if txt=='ne':
            return 2/8
        if txt=='ene' or txt=='nee':
            return 3/8
        if txt=='e':
            return 4/8
        if txt=='ese' or txt=='see':
            return 5/8
        if txt=='se':
            return 6/8
        if txt=='ses' or txt=='sse':
            return 7/8
        if txt=='s':
            return 8/8
        if txt=='ssw' or txt=='sws':
            return 9/8
        if txt=='sw':
            return 10/8
        if txt=='sww' or txt=='wsw':
            return 11/8
        if txt=='w':
            return 12/8
        if txt=='wnw' or txt=='nww':
            return 13/8
        if txt=='nw':
            return 14/8
        if txt=='nwn' or txt=='nnw':
            return 15/8
        return np.nan
    except:
        return np.nan
train_single['WindDirection'] = train_single['WindDirection'].apply(transform_WindDirection)

In [37]:
train_single['S_horizontal_qb'] = np.cos(train_single.QB_dir) * train_single.QB_S
train_single['S_vertical_qb'] = np.sin(train_single.QB_dir) * train_single.QB_S
train_single['S_h_diff'] = np.abs(train_single['S_horizontal_qb'] - train_single.S_horizontal)
train_single['S_v_diff'] = np.abs(train_single['S_vertical_qb'] - np.sin(train_single.Dir) * train_single.S)
train_single.drop(['QB_S','QB_dir'],axis=1,inplace=True)

In [38]:
playid1 = train_single.PlayId.copy()

In [39]:
train_single123 = train_single.copy()

In [40]:
train_single.drop(remove_features,axis=1,inplace=True)

In [41]:
drop_list222 = ['S_5_def_qb',
 'dist_5_off',
 'A_2_off',
 'A_3_off',
 'A_5_def',
 'S_10_def_mean',
 'S_3_def',
 'S_5_def',
 'A_4_def',
 'A_4_off',
 'S_3_off_qb',
 'S_vertical_qb',
 'S_5_def_QB_mean',
 'S_5_off',
 'S_2_def',
 'PlayerCollegeName',
 'dist_3_off_qb',
 'dist_5_off_qb',
 'Stadium',
 'Temperature',
 'std_all_Y',
 'dist_4_off',
 'Location',
 'Season',
 'S_1_def',
 'PlayerWeight',
 'Week',
 'WindSpeed',
 'runner_height',
 'num_dist5_off',
 'PlayDirection',
 'OffenseFormation',
 'off_num_WR',
 'StadiumType',
 'Quarter',
 'num_dist5_def_QB',
 'Def_num_DL',
 'off_num_OL',
 'num_dist2_off',
 'Def_num_OL',
 'off_num_DB',
 'off_num_QB',
 'off_num_LB',
 'off_num_DL',
 'qb_number']
train_single.drop(drop_list222,axis=1,inplace=True)

In [42]:
temp1 = ['run_y',  'dist_2_def', 'A_2_def', 'A_5_off_mean','A','A_1_off_qb']

train_single.drop(['A_2_off_QB_mean','A_5_off_QB_mean','A_5_def_QB_mean','A_10_def_QB_mean','A_10_off_QB_mean','A_5_def_qb','A_4_def_qb','A_4_off_qb','A_3_def_qb','A_3_off_qb','A_1_def_qb','A_2_def_qb','A_2_off_qb','A_5_off_qb'],axis=1,inplace=True)
train_single.drop(temp1,axis=1,inplace=True)


In [43]:
train_single_nn = train_single.copy()

In [44]:
train_single.fillna(-999,inplace=True)

In [45]:
train_single.shape

(31008, 120)

In [52]:
y_train = train_single.Yards
X_train = train_single.drop(['Yards'],axis=1)
X_train_123 = X_train.copy()
for f in X_train.columns:
    if X_train[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f])+[-999])
        X_train[f] = lbl.transform(list(X_train[f]))

In [53]:
bins=[-100,-6,-3,-1,0,1,2,3,4,5,7,10,13,16,21,26,36,50,99]
def y_to_cate(y_train,bins):
    k = 101
    y_train_transform = y_train.copy()
    for i in range(len(bins)-1):
        y_train_transform[(y_train_transform>bins[i])&(y_train_transform<=bins[i+1])] = k
        k +=1
    return y_train_transform-101
def cal_CPRS(y_true_array,y_pred_array):
    return np.mean((y_true_array - y_pred_array)**2)
def raw_y_to_array2(pred,num_counts,bins,dist_to_ends):
    y_pred_array = np.zeros((pred.shape[0],199))
    n1 = pred.shape[0]
    n = len(bins)-1
    for j in range(n):
        temp = (num_counts.loc[list(range(bins[j]+1,bins[j+1]+1))])
        temp1 = list(temp/np.sum(temp))
        temp_s = np.tile(temp1,(n1,1))
        temp_block = (pred[:,j]*temp_s.transpose()).transpose()
        y_pred_array[:,(bins[j]+100):(bins[j+1]+100)] = temp_block.copy()
    temp_array = np.clip(np.cumsum(y_pred_array,axis=1),0,1)
    for i in range(temp_array.shape[0]):
        temp_array[i,(dist_to_ends.iloc[i]+99):] = 1
    return temp_array
def raw_y_to_array2_test(pred,num_counts,bins,dist_to_ends):
    y_pred_array = np.zeros((1,199))
    pred = pred.reshape(1,-1)
    n1 = pred.shape[0]
    n = len(bins)-1
    num_counts_copy = num_counts.copy()
    num_counts_copy.loc[list(range(dist_to_ends.values[0]+1,100))] = 0
    for j in range(n-1,-1,-1):
        temp = (num_counts_copy.loc[list(range(bins[j]+1,bins[j+1]+1))])
        if np.sum(temp.values)==0:
            #print('all zeros happens')
            pred[:,j-1] += pred[:,j] 
            temp1 = list(temp).copy()
        else:
            temp1 = list(temp/np.sum(temp))
        temp_s = np.tile(temp1,(n1,1))
        temp_block = (pred[:,j]*temp_s.transpose()).transpose()
        y_pred_array[:,(bins[j]+100):(bins[j+1]+100)] = temp_block.copy()
    temp_array = np.clip(np.cumsum(y_pred_array,axis=1),0,1)
    temp_array[0,(dist_to_ends.values[0]+99):] = 1
    return temp_array
def y_true_array(y_true):
    temp = np.zeros((y_true.shape[0],199))
    for i,value in enumerate(y_true):
        temp[i,(value+99):] = 1
    return temp

In [54]:
def CRPS_eval(y_true, y_prob):
    y_prob =  y_prob.reshape((len(y_true),-1),order='F')
    global a
    a=y_prob.copy()
    

    if len(y_true) < y_train.shape[0]/3:
        index1 = test_index.copy()
    else:
        index1 = train_index.copy()
    y_pred_array1 = raw_y_to_array2(y_prob,num_counts,bins,train_single['dist_to_end'].iloc[index1])
    return 'CRPS', cal_CPRS(y_true_array1[index1,:],y_pred_array1), False

In [55]:
print(train_single.shape)

(31008, 120)


In [56]:
num_counts = y_train.value_counts().copy()
for i in range(-99,100):
    if i not in set(num_counts.index):
        num_counts[i] = 0

In [ ]:
kf=KFold(n_splits = 5)
impor1 = 0
resu1_logloss = 0
resu2_cprs = 0
models = []
y_train_trans = y_to_cate(y_train,bins)
y_true_array1 = y_true_array(y_train)
oof_predict = np.zeros((y_train.shape[0],18))
for train_index, test_index in kf.split(X_train, y_train_trans):
    X_train2= X_train.iloc[train_index,:]
    y_train2= y_train_trans.iloc[train_index]
    X_test2= X_train.iloc[test_index,:]
    y_test2= y_train_trans.iloc[test_index]

    clf = lgb.LGBMClassifier(n_estimators=10000, random_state=47,learning_rate=0.01,importance_type = 'gain',
                     n_jobs = -1,num_leaves=20,bagging_freq=1,colsample_bytree=0.5,subsample=1,min_child_weight = 0.1,
                             min_child_samples = 250,reg_alpha = 1.5,reg_lambda = 1,metric = 'None')
    clf.fit(X_train2,y_train2,eval_set = [(X_test2,y_test2)],early_stopping_rounds=200,verbose=50,eval_metric = CRPS_eval)
    models.append(clf)
    temp_predict_prob = clf.predict_proba(X_test2)
    oof_predict[test_index,:] = temp_predict_prob
    crps = cal_CPRS(y_true_array1[test_index,:],raw_y_to_array2(temp_predict_prob,num_counts,bins,train_single['dist_to_end'].iloc[test_index]))
    print(crps)
    resu1_logloss += log_loss(y_test2,temp_predict_prob)/5
    resu2_cprs += crps/5
    impor1 += clf.feature_importances_/5
    gc.collect()
print('mean cprs:',resu2_cprs)
print('oof cprs:',cal_CPRS(y_true_array1,raw_y_to_array2(oof_predict,num_counts,bins,train_single['dist_to_end'])))
print('mean logloss',resu1_logloss)
print('oof logloss',log_loss(y_train_trans,oof_predict))

Training until validation scores don't improve for 200 rounds.
[50]	valid_0's CRPS: 0.013294
[100]	valid_0's CRPS: 0.0130854
[150]	valid_0's CRPS: 0.0129239
[200]	valid_0's CRPS: 0.0127968
[250]	valid_0's CRPS: 0.0126971
[300]	valid_0's CRPS: 0.0126183
[350]	valid_0's CRPS: 0.0125514
[400]	valid_0's CRPS: 0.0124963
[450]	valid_0's CRPS: 0.0124511
[500]	valid_0's CRPS: 0.012414
[550]	valid_0's CRPS: 0.0123815
[600]	valid_0's CRPS: 0.0123544
[650]	valid_0's CRPS: 0.0123304
[700]	valid_0's CRPS: 0.012309
[750]	valid_0's CRPS: 0.0122887
[800]	valid_0's CRPS: 0.0122727
[850]	valid_0's CRPS: 0.0122582
[900]	valid_0's CRPS: 0.0122458
[950]	valid_0's CRPS: 0.0122346
[1000]	valid_0's CRPS: 0.0122263
[1050]	valid_0's CRPS: 0.0122177
[1100]	valid_0's CRPS: 0.0122095
[1150]	valid_0's CRPS: 0.0122021
[1200]	valid_0's CRPS: 0.0121959
[1250]	valid_0's CRPS: 0.0121904
[1300]	valid_0's CRPS: 0.0121853
[1350]	valid_0's CRPS: 0.0121803
[1400]	valid_0's CRPS: 0.0121762
[1450]	valid_0's CRPS: 0.0121718
[15

### NN part

In [50]:
import os
TRAIN_ABLE_FALSE=True
if TRAIN_ABLE_FALSE:
    os.environ['CUDA_VISIBLE_DEVICES'] = "1"
import numpy as np
import pandas as pd
import sklearn.metrics as mtr
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda,BatchNormalization
from keras.models import Model
import keras.backend as K
from keras.callbacks import Callback
from  keras.callbacks import EarlyStopping,ModelCheckpoint
import datetime
from scipy.stats import mode

Using TensorFlow backend.


In [51]:
cat = ['Team','NflId','Down','OffensePersonnel','DefensePersonnel','Position','GameWeather']
for column in train_single_nn.columns:
    if column not in cat:
        if pd.isna(train_single_nn[column].median()):
            train_single_nn[column] = train_single_nn[column].fillna(-1)
            #print('???????????????????????????????????????????????')
        else:
            #print(train_single_nn[column].median())
            train_single_nn[column] = train_single_nn[column].fillna(train_single_nn[column].median())
    else:
        train_single_nn[column] = train_single_nn[column].fillna(mode(train_single_nn[column])[0][0])

/opt/conda/lib/python3.6/site-packages/scipy/stats/stats.py:248: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)
/opt/conda/lib/python3.6/site-packages/scipy/stats/stats.py:248: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)
/opt/conda/lib/python3.6/site-packages/scipy/stats/stats.py:248: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [52]:
train_single_nn.shape

(23171, 120)

In [53]:
y_train_nn = train_single_nn.Yards
X_train_nn = train_single_nn.drop(['Yards'],axis=1)
X_train_123_nn = X_train_nn.copy()
for f in X_train_nn.columns:
    if X_train_nn[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train_nn[f]))
        X_train_nn[f] = lbl.transform(list(X_train_nn[f]))

In [54]:
import sklearn.metrics as mtr
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda,BatchNormalization
from keras.models import Model
import keras.backend as K
from keras.callbacks import Callback
from  keras.callbacks import EarlyStopping,ModelCheckpoint
import datetime

In [55]:
yards = train_single_nn.Yards
X = X_train_nn.copy()
y = np.zeros((yards.shape[0], 199))
for idx, target in enumerate(list(yards)):
    y[idx][99 + target] = 1
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [56]:
from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda
from keras.models import Model
import keras.backend as K
import re
from keras.losses import binary_crossentropy
from  keras.callbacks import EarlyStopping,ModelCheckpoint
import codecs

from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from sklearn.metrics import f1_score




class CRPSCallback(Callback):
    
    def __init__(self,validation, predict_batch_size=20, include_on_batch=False):
        super(CRPSCallback, self).__init__()
        self.validation = validation
        self.predict_batch_size = predict_batch_size
        self.include_on_batch = include_on_batch
        
        print('validation shape',len(self.validation))

    def on_batch_begin(self, batch, logs={}):
        pass

    def on_train_begin(self, logs={}):
        if not ('CRPS_score_val' in self.params['metrics']):
            self.params['metrics'].append('CRPS_score_val')

    def on_batch_end(self, batch, logs={}):
        if (self.include_on_batch):
            logs['CRPS_score_val'] = float('-inf')

    def on_epoch_end(self, epoch, logs={}):
        logs['CRPS_score_val'] = float('-inf')
            
        if (self.validation):
            X_valid, y_valid = self.validation[0], self.validation[1]
            y_pred = self.model.predict(X_valid)
            y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
            y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
            val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_valid.shape[0])
            val_s = np.round(val_s, 6)
            logs['CRPS_score_val'] = val_s
    


In [57]:
def get_model(x_tr,y_tr,x_val,y_val):
    inp = Input(shape = (x_tr.shape[1],))
    x = Dense(1024, input_dim=X.shape[1], activation='relu')(inp)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    
    out = Dense(199, activation='softmax')(x)
    model = Model(inp,out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[])
    #add lookahead
#     lookahead = Lookahead(k=5, alpha=0.5) # Initialize Lookahead
#     lookahead.inject(model) # add into model

    
    es = EarlyStopping(monitor='CRPS_score_val', 
                       mode='min',
                       restore_best_weights=True, 
                       verbose=1, 
                       patience=10)

    mc = ModelCheckpoint('best_model.h5',monitor='CRPS_score_val',mode='min',
                                   save_best_only=True, verbose=1, save_weights_only=True)
    
    bsz = 1024
    steps = x_tr.shape[0]/bsz
    


    model.fit(x_tr, y_tr,callbacks=[CRPSCallback(validation = (x_val,y_val)),es,mc], epochs=100, batch_size=bsz,verbose=1)
    model.load_weights("best_model.h5")
    
    y_pred = model.predict(x_val)
    y_valid = y_val
    y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
    val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * x_val.shape[0])
    crps = np.round(val_s, 6)

    return model,crps

In [58]:
from sklearn.model_selection import train_test_split, KFold
import time

losses = []
models_nn = []
crps_csv = []
#oof3_nn=[]

s_time = time.time()


for k in range(1):
    kfold = KFold(5, random_state = 42 + k, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(yards)):
        print("-----------")
        print("-----------")
        tr_x,tr_y = X[tr_inds],y[tr_inds]
        val_x,val_y = X[val_inds],y[val_inds]
        model,crps = get_model(tr_x,tr_y,val_x,val_y)
        #oof3_nn.append(oof)
        models_nn.append(model)
        print("the %d fold crps is %f"%((k_fold+1),crps))
        crps_csv.append(crps)
 
print("mean crps is %f"%np.mean(crps_csv))


def predict(x_te):
    model_num = len(models_nn)
    for k,m in enumerate(models_nn):
        if k==0:
            y_pred = m.predict(x_te,batch_size=1024)
        else:
            y_pred+=m.predict(x_te,batch_size=1024)
            
    y_pred = y_pred / model_num
    
    return y_pred
            

-----------
-----------
validation shape 2
Epoch 1/100
18536/18536 [==============================] - 3s 174us/step - loss: 5.5721

Epoch 00001: CRPS_score_val improved from inf to 0.08046, saving model to best_model.h5
Epoch 2/100
18536/18536 [==============================] - 2s 108us/step - loss: 5.0604

Epoch 00002: CRPS_score_val improved from 0.08046 to 0.07575, saving model to best_model.h5
Epoch 3/100
18536/18536 [==============================] - 2s 108us/step - loss: 4.6347

Epoch 00003: CRPS_score_val improved from 0.07575 to 0.06516, saving model to best_model.h5
Epoch 4/100
18536/18536 [==============================] - 2s 106us/step - loss: 4.2335

Epoch 00004: CRPS_score_val improved from 0.06516 to 0.04904, saving model to best_model.h5
Epoch 5/100
18536/18536 [==============================] - 2s 107us/step - loss: 3.8559

Epoch 00005: CRPS_score_val improved from 0.04904 to 0.02846, saving model to best_model.h5
Epoch 6/100
18536/18536 [==============================]

### submission

In [59]:

def transform_test(train):
    train.loc[train.VisitorTeamAbbr == "ARI",'VisitorTeamAbbr'] = "ARZ"
    train.loc[train.HomeTeamAbbr == "ARI",'HomeTeamAbbr'] = "ARZ"

    train.loc[train.VisitorTeamAbbr == "BAL",'VisitorTeamAbbr'] = "BLT"
    train.loc[train.HomeTeamAbbr == "BAL",'HomeTeamAbbr'] = "BLT"

    train.loc[train.VisitorTeamAbbr == "CLE",'VisitorTeamAbbr'] = "CLV"
    train.loc[train.HomeTeamAbbr == "CLE",'HomeTeamAbbr'] = "CLV"

    train.loc[train.VisitorTeamAbbr == "HOU",'VisitorTeamAbbr'] = "HST"
    train.loc[train.HomeTeamAbbr == "HOU",'HomeTeamAbbr'] = "HST"    
    train['ToLeft'] = train.PlayDirection == "left"
    train['X_std'] = train.X
    train.loc[train.ToLeft, 'X_std'] = 120 - train.loc[train.ToLeft, 'X'] 
    train['X'] = train['X_std'].copy()
    train['Y_std'] = train.Y
    train.loc[train.ToLeft, 'Y_std'] = 160/3 - train.loc[train.ToLeft, 'Y'] 
    train['Y'] = train['Y_std'].copy()
    train['Orientation_std'] = np.mod(train.Orientation-90, 360)
    train.loc[train.ToLeft, 'Orientation_std'] = np.mod(180 + train.loc[train.ToLeft, 'Orientation_std'].copy(),360)
    train['Orientation'] = train['Orientation_std'].copy()
    train['Dir_rad'] = np.mod(train.Dir-90, 360) * np.pi/180.0
    train['Dir_std'] = train.Dir_rad.copy()
    train.loc[train.ToLeft, 'Dir_std'] = np.mod(np.pi + train.loc[train.ToLeft, 'Dir_rad'], 2*np.pi)
    train['Dir'] = train['Dir_std'].copy()
    person_info = train[person_feature].copy()
    person_info['player_mark'] = ['player'+ str(i) for i in range(1,12)]*int(person_info.shape[0]/11)
    te = pd.pivot_table(person_info,columns=['Team','player_mark'],index=['PlayId'],aggfunc=lambda x:x)
    te.columns = ['_'.join(a) for a in te.columns.values]
    train['TeamOnOffense'] = "home"
    train.loc[train.PossessionTeam != train.HomeTeamAbbr, 'TeamOnOffense'] = "away"
    train['IsOnOffense'] = train.Team == train.TeamOnOffense
    
    train['is_run'] = train.NflId == train.NflIdRusher
    train_single = train[train.is_run==True].copy()
    train_single['diff_Ori_dir'] = np.abs(train_single.Orientation - train_single.Dir/np.pi*180)
    temp1 = pd.DataFrame(train_single['diff_Ori_dir'].copy())
    temp1['c1'] = 360 - temp1.diff_Ori_dir
    train_single['diff_Ori_dir']= np.min(temp1,axis=1).tolist()
    train_single['time_quarter'] = train_single.GameClock.map(lambda x:transform_time_quarter(x))
    train_single['time_end'] = train_single.apply(lambda x:transform_time_all(x.loc['GameClock'],x.loc['Quarter']),axis=1)
    train_single['TimeHandoff'] = train_single['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    train_single['TimeSnap'] = train_single['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    train_single['date_game'] = train_single.GameId.map(lambda x:pd.to_datetime(str(x)[:8]))
    train_single['runner_age'] = (train_single.date_game.map(pd.to_datetime) - train_single.PlayerBirthDate.map(pd.to_datetime)).map(lambda x:x.days)/365.25
    train_single['runner_height'] = train_single.PlayerHeight.map(transform_height)
    train_single['own_field'] = (train_single['FieldPosition'] == train_single['PossessionTeam']).astype(int)
    train_single['dist_to_end'] = train_single.apply(lambda x:(100 - x.loc['YardLine']) if x.loc['own_field']==1 else x.loc['YardLine'],axis=1)
    train_single['YardLine_x'] = train_single.apply(transform_YardLine_x,axis=1)
    train_single['dist_to_line'] = train_single.apply(transform_dist_to_line,axis=1)
    train['is_QB'] = train.Position=='QB'
    qb_number = train.groupby(['PlayId'])['is_QB'].sum()
    train_single['qb_number'] = train_single.PlayId.map(qb_number)
    te1 = train.loc[(train.is_run==True)|(train.is_QB==1),['PlayId','X','Y','is_run','is_QB','A','S',
                                                           'Dis','Orientation','Dir','PlayerHeight','PlayerWeight','PlayerBirthDate']]
    te1 = te1.set_index(['PlayId'])
    run_info = te1[te1.is_run==True].copy()
    run_info.rename({'X':'run_x','Y':'run_y'},axis=1,inplace=True)
    qb_info = te1[te1.is_QB==1].copy()
    qb_info.reset_index(inplace=True)
    
    drop_index=[]
    for num_index in qb_number.index[qb_number>=2]:
        temp = qb_info.loc[qb_info.PlayId==num_index,['X','Y']]
        dist_to_close = []
        for num in temp.index:
            temp_a = np.sqrt(np.sum((temp.loc[num].values - run_info.loc[run_info.index==num_index,['run_x','run_y']].values[0,])**2))
            #print(num_index)
            dist_to_close.append(temp_a)
        drop_index.extend([temp.index.tolist()[i] for i in list(np.argsort(dist_to_close))[1:]])
    qb_info.drop(drop_index,axis=0,inplace=True)
#     drop_index=[]
#     for num_index in qb_number.index[qb_number==2]:
#         temp = qb_info.loc[qb_info.PlayId==num_index,['X','Y']]
#         dist_to_close = []
#         for num in temp.index:
#             temp_a = np.sqrt(np.sum((temp.loc[num].values - run_info.loc[run_info.index==num_index,['run_x','run_y']].values[0,])**2))
#             #print(num_index)
#             dist_to_close.append(temp_a)
#         if dist_to_close[0]>dist_to_close[1]:
#             drop_index.append(temp.index.tolist()[0])
#         else:
#             drop_index.append(temp.index.tolist()[1])

    train_single.rename({'X':'run_x','Y':'run_y'},axis=1,inplace=True)
    qb_info.rename({'A':'QB_A','S':'QB_S','Dis':'QB_Dis','Orientation':'QB_Orientation',
                    'Dir':'QB_dir','PlayerHeight':'QB_PlayerHeight',
                    'PlayerWeight':'QB_PlayerWeight','PlayerBirthDate':'QB_PlayerBirthDate'},axis=1,inplace=True)
    train_single = pd.merge(train_single,qb_info[['PlayId','X','Y','QB_A','QB_S','QB_Dis','QB_Orientation',
                                                 'QB_dir','QB_PlayerHeight','QB_PlayerWeight',
                                                 'QB_PlayerBirthDate']],on='PlayId',how='left')
    train_single.rename({'X':'QB_x','Y':'QB_y'},axis=1,inplace=True)
    train_single['qb_to_line'] = train_single.apply(transform_qb_to_line,axis=1)

    train_single['run_dist_qb'] = train_single.apply(transform_run_dist_qb,axis=1)
    train_single['QB_PlayerHeight'] = train_single.QB_PlayerHeight.map(transform_height)
    train_single['QB_age'] = (train_single.date_game.map(pd.to_datetime) - train_single.QB_PlayerBirthDate.map(pd.to_datetime)).map(lambda x:x.days)/365.25

    train_single['diff_Ori_dir_qb'] = np.abs(train_single.QB_Orientation - train_single.QB_dir/np.pi*180)
    temp1 = pd.DataFrame(train_single['diff_Ori_dir_qb'].copy())
    temp1['c1'] = 360 - temp1.diff_Ori_dir_qb
    train_single['diff_Ori_dir_qb']= np.min(temp1,axis=1).tolist()

    train_single['diff_qb_runner_dir'] = np.abs(train_single.Dir/np.pi*180 - train_single.QB_dir/np.pi*180)
    temp1 = pd.DataFrame(train_single['diff_qb_runner_dir'].copy())
    temp1['c1'] = 360 - temp1.diff_qb_runner_dir
    train_single['diff_qb_runner_dir']= np.min(temp1,axis=1).tolist()

    train_single['diff_qb_runner_ori'] = np.abs(train_single.Orientation - train_single.QB_Orientation)
    temp1 = pd.DataFrame(train_single['diff_qb_runner_ori'].copy())
    temp1['c1'] = 360 - temp1.diff_qb_runner_ori
    train_single['diff_qb_runner_ori']= np.min(temp1,axis=1).tolist()

    train_single['score_difference'] = train_single.apply(lambda x:
                                        x.loc['HomeScoreBeforePlay'] - x.loc['VisitorScoreBeforePlay'] if x.loc['PossessionTeam'] == x.loc['HomeTeamAbbr']
                                                          else x.loc['VisitorScoreBeforePlay'] - x.loc['HomeScoreBeforePlay'],axis=1)
    train_single['offense_score'] = train_single.apply(lambda x:
                                        x.loc['HomeScoreBeforePlay']  if x.loc['PossessionTeam'] == x.loc['HomeTeamAbbr']
                                                          else x.loc['VisitorScoreBeforePlay'] ,axis=1)
    
    train_single = pd.merge(train_single,te,on='PlayId',how = 'left')

    train_single['dist1_divide_S'] = train_single.apply(dist1_divide_S,axis=1)

    train_single['dist1_divide_S_hor'] = train_single.apply(dist1_divide_S_hor,axis=1)

    train_single['time1_dist_S'] = train_single.apply(time1_dist_S,axis=1)

    train_single['time1_dist_S_hor'] = train_single.apply(time1_dist_S_hor,axis=1)

    train_single['close_who_S_ratio']= train_single.apply(transform_close_who,axis=1)
    
    temp1  = train_single.apply(transform_dist_close123_offense_runner,axis=1)
    names = ['dist_1_off','A_1_off','S_1_off','dist_2_off','A_2_off','S_2_off','dist_3_off','A_3_off','S_3_off',
            'dist_4_off','A_4_off','S_4_off','dist_5_off','A_5_off','S_5_off']
    for i in range(len(names)):
        train_single[names[i]] = temp1.map(lambda x:x[i])

    temp1  = train_single.apply(transform_dist_close123_defender_runner,axis=1)
    names = ['dist_1_def','A_1_def','S_1_def','dist_2_def','A_2_def','S_2_def','dist_3_def','A_3_def','S_3_def',
            'dist_4_def','A_4_def','S_4_def','dist_5_def','A_5_def','S_5_def']
    for i in range(len(names)):
        train_single[names[i]] = temp1.map(lambda x:x[i])
    for i in [1]:
        train_single['dist_'+str(i)+'_S_ratio'] = train_single['S_'+str(i)+'_def'] / train_single.S
        train_single['dist_'+str(i)+'_S_ratio'] = np.where(np.isinf(train_single['dist_'+str(i)+'_S_ratio']),999,train_single['dist_'+str(i)+'_S_ratio'])
        
        
    temp1  = train_single.apply(transform_dist_close123_offense_QB,axis=1)
    names = ['dist_1_off_qb','A_1_off_qb','S_1_off_qb','dist_2_off_qb','A_2_off_qb','S_2_off_qb','dist_3_off_qb','A_3_off_qb','S_3_off_qb',
            'dist_4_off_qb','A_4_off_qb','S_4_off_qb','dist_5_off_qb','A_5_off_qb','S_5_off_qb']
    for i in range(len(names)):
        train_single[names[i]] = temp1.map(lambda x:x[i])

    temp1  = train_single.apply(transform_dist_close123_defender_QB,axis=1)
    names = ['dist_1_def_qb','A_1_def_qb','S_1_def_qb','dist_2_def_qb','A_2_def_qb','S_2_def_qb','dist_3_def_qb','A_3_def_qb','S_3_def_qb',
            'dist_4_def_qb','A_4_def_qb','S_4_def_qb','dist_5_def_qb','A_5_def_qb','S_5_def_qb']
    for i in range(len(names)):
        train_single[names[i]] = temp1.map(lambda x:x[i])
        
    temp1  = train_single.apply(transform_dist2_num_defender,axis=1)
    names = ['num_dist5_def','S_5_def_mean','A_5_def_mean','num_dist10_def','S_10_def_mean','A_10_def_mean']
    for i in range(len(names)):
        train_single[names[i]] = temp1.map(lambda x:x[i])

    temp1  = train_single.apply(transform_dist2_num_offense,axis=1)
    names = ['num_dist2_off','S_2_off_mean','A_2_off_mean','num_dist5_off','S_5_off_mean','A_5_off_mean','num_dist10_off','S_10_off_mean','A_10_off_mean']
    for i in range(len(names)):
        train_single[names[i]] = temp1.map(lambda x:x[i])
        
    temp1  = train_single.apply(transform_dist2_num_defender_QB,axis=1)
    names = ['num_dist5_def_QB','S_5_def_QB_mean','A_5_def_QB_mean','num_dist10_def_QB','S_10_def_QB_mean','A_10_def_QB_mean']
    for i in range(len(names)):
        train_single[names[i]] = temp1.map(lambda x:x[i])

    temp1  = train_single.apply(transform_dist2_num_offense_QB,axis=1)
    names = ['num_dist2_off_QB','S_2_off_QB_mean','A_2_off_QB_mean','num_dist5_off_QB','S_5_off_QB_mean','A_5_off_QB_mean','num_dist10_off_QB','S_10_off_QB_mean','A_10_off_QB_mean']
    for i in range(len(names)):
        train_single[names[i]] = temp1.map(lambda x:x[i])
    temp1  = train_single.apply(transform_defend_X_Y_std,axis=1)
    names = ['std_def_X','std_def_Y']
    for i in range(len(names)):
        train_single[names[i]] = temp1.map(lambda x:x[i])

    temp1  = train_single.apply(transform_offense_X_Y_std,axis=1)
    names = ['std_off_X','std_off_Y']
    for i in range(len(names)):
        train_single[names[i]] = temp1.map(lambda x:x[i])

    temp1  = train_single.apply(transform_all_X_Y_std,axis=1)
    names = ['std_all_X','std_all_Y']
    for i in range(len(names)):
        train_single[names[i]] = temp1.map(lambda x:x[i])

    temp1 = train_single.OffensePersonnel.apply(lambda x: pd.Series(OffensePersonnelSplit(x)))
    names = ['DB','DL','LB','OL','QB','RB','TE','WR']
    for i in range(len(names)):
        train_single['off_num_' + names[i]] = temp1.apply(lambda x:x[names[i]],axis=1)

    temp1 = train_single.DefensePersonnel.apply(lambda x: pd.Series(DefensePersonnelSplit(x)))
    names = ['DB','DL','LB','OL']
    for i in range(len(names)):
        train_single['Def_num_' + names[i]] = temp1.apply(lambda x:x[names[i]],axis=1)

    train_single['S_horizontal'] = np.cos(train_single.Dir) * train_single.S
    train_single['S_vertical'] = np.abs(np.sin(train_single.Dir) * train_single.S)
    
    def is_in_poly(polygon,array):
        try:
            k = 0
            temp = Delaunay(polygon)
            for i in range(array.shape[0]):
                if temp.find_simplex(array[i,:]) >=0:
                    k+=1
            return k
        except:
            return 0

    def calculate_voroni(row):
        try:
            df=train[train.PlayId==row.PlayId]
            df_offense = df[(df.NflIdRusher != df.NflId)&(df.IsOnOffense)].copy()
            pp = df_offense[['X_std','Y_std']].values.copy()
            df = df[(df.NflIdRusher == df.NflId) | ~df.IsOnOffense] 
            xy = df[['X_std', 'Y_std']].values
            n_points = xy.shape[0]
            xy1 = xy.copy()
            xy1[:,1] = - xy[:,1]
            xy2 = xy.copy()
            xy2[:,1] = 320/3 - xy[:,1]
            xy3 = xy.copy()
            xy3[:,0] =  - xy[:,0]
            xy4 = xy.copy()
            xy4[:,0] = 240 - xy[:,0]
            xy = np.concatenate((xy, xy1, xy2, xy3, xy4), axis=0)
            offense = df.IsOnOffense.values
            vor = Voronoi(xy)
            num_adj = []
            adj_vertice = []
            X_temp = []
            Y_temp = []
            S_temp = []
            Dir_temp = []
            for r in range(n_points):
                if offense[r]:
                    rusher_num = r
                    rusher_region_points = vor.regions[vor.point_region[r]]
            for r in range(n_points):
                region = vor.regions[vor.point_region[r]]
                if not -1 in region:
                    polygon = [vor.vertices[i] for i in region]
                    if offense[r]:
                        temp_area = ConvexHull(polygon)
                        voroni_area = temp_area.volume
                        x_max = np.max(np.array(polygon)[:,0])
                        x_min = np.min(np.array(polygon)[:,0])
                    else:
                        if len(set(rusher_region_points).intersection(set(region)))>=1:
                            num_adj.append(r)
                            adj_vertice.append(polygon)
            X_temp = df.X_std.values.copy()[num_adj]
            X_Y_temp = df[['X_std','Y_std']].values.copy()[num_adj,:]
            S_temp = df.S.values.copy()[num_adj]
            Dir_temp = df.Dir.values.copy()[num_adj]
            num_in_adj_cell = [is_in_poly(np.array(poly),pp) for poly in adj_vertice]
            num_0_cell = len(list(filter(lambda x:x==0,num_in_adj_cell)))
            sum_num_adj_cell = np.sum(num_in_adj_cell)

            if num_0_cell==0:
                dist1 = np.nan
                S_ratio = np.nan
                time1 = np.nan
                time1_h = np.nan
            else:
                #dist1 = np.min(np.sqrt(np.sum((X_Y_temp[list(np.array(num_in_adj_cell)==0),:] - np.array([row.run_x,row.run_y]))**2,axis=1)))
                S_ratio = np.clip(np.max(S_temp[list(np.array(num_in_adj_cell)==0)]/row.S),0,300)
                #time1 = np.clip(np.min(np.sqrt(np.sum((X_Y_temp[list(np.array(num_in_adj_cell)==0),:] - np.array([row.run_x,row.run_y]))**2,axis=1))/S_temp[list(np.array(num_in_adj_cell)==0)]),0,500)
                time1_h = np.clip(np.min(np.abs(X_temp[list(np.array(num_in_adj_cell)==0)] - row.run_x)/np.abs(S_temp[list(np.array(num_in_adj_cell)==0)]*np.cos(Dir_temp[list(np.array(num_in_adj_cell)==0)]))),0,500)
            return voroni_area,110- x_max,x_max-row.run_x, x_max - row.YardLine_x,len(adj_vertice),sum_num_adj_cell,num_0_cell,S_ratio,time1_h
        except:
            return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan


    te = train_single.apply(calculate_voroni,axis=1)
    names = ['voroni_area','voroni_bc_close','voroni_bc_x_max_minus_x','voroni_max_yardline','adj_cell_num','adj_off_num','adj_def_num_no_offense','voro0_S_ratio_max','voro0_time_min_h']
    for i in range(len(names)):
        train_single[names[i]] = te.map(lambda x:x[i])
    train_single['voro_diff_num'] = train_single['adj_cell_num'] - train_single['adj_off_num']
    

    
    ### player info
    train_single['mean_player_100'] = 0
    train_single['std_player_100'] = 0
    train_single['min_player_100'] = 0
    train_single['quan25_player_100'] = 0
    train_single['quan75_player_100'] = 0
    for player in list(train_single.NflId.unique()):
        temp = train_single123.loc[train_single123.NflId==player,'Yards']
        index1 = train_single[train_single.NflId==player].index
        if temp.shape[0] == 0:
            train_single.loc[index1,'mean_player_100'] = np.nan
            train_single.loc[index1,'std_player_100'] = np.nan
            train_single.loc[index1,'min_player_100'] = np.nan
            train_single.loc[index1,'quan25_player_100'] = np.nan
            train_single.loc[index1,'quan75_player_100'] = np.nan
        else:     
            mean1 = temp.rolling(100,min_periods=20).mean().iloc[-1]
            std1 = temp.rolling(100,min_periods=20).std().iloc[-1]
            min1 = temp.rolling(100,min_periods=20).min().iloc[-1]
            quan25 = temp.rolling(100,min_periods=20).quantile(0.10).iloc[-1]
            quan75 = temp.rolling(100,min_periods=20).quantile(0.90).iloc[-1]      
            train_single.loc[index1,'mean_player_100'] = mean1
            train_single.loc[index1,'std_player_100'] = std1
            train_single.loc[index1,'min_player_100'] = min1
            train_single.loc[index1,'quan25_player_100'] = quan25
            train_single.loc[index1,'quan75_player_100'] = quan75

    train_single['DefendersInTheBox_vs_Distance'] = train_single['DefendersInTheBox'] / train_single['Distance']

    train_single['WindDirection'] = train_single['WindDirection'].apply(clean_WindDirection)

    train_single['WindDirection'] = train_single['WindDirection'].apply(transform_WindDirection)
    train_single['S_horizontal_qb'] = np.cos(train_single.QB_dir) * train_single.QB_S
    train_single['S_vertical_qb'] = np.sin(train_single.QB_dir) * train_single.QB_S
    train_single['S_h_diff'] = np.abs(train_single['S_horizontal_qb'] - train_single.S_horizontal)
    train_single['S_v_diff'] = np.abs(train_single['S_vertical_qb'] - np.sin(train_single.Dir) * train_single.S)
    train_single.drop(['QB_S','QB_dir'],axis=1,inplace=True)


    temp1 = ['run_y', 'dist_2_def', 'A_2_def', 'A_5_off_mean','A','A_1_off_qb']
    train_single.drop(['A_2_off_QB_mean','A_5_off_QB_mean','A_5_def_QB_mean','A_10_def_QB_mean','A_10_off_QB_mean','A_5_def_qb','A_4_def_qb','A_4_off_qb','A_3_def_qb','A_3_off_qb','A_1_def_qb','A_2_def_qb','A_2_off_qb','A_5_off_qb'],axis=1,inplace=True)
    train_single = train_single.drop(temp1,axis=1)
    train_single.drop(drop_list222,axis=1,inplace=True)    

    return train_single.drop(remove_features,axis=1)

In [60]:
import time

time_start=time.time()


for (test_df, sample_prediction_df) in env.iter_test():
    ### tree part
    X_test = transform_test(test_df)
    basetable = X_test.copy()
    X_test.fillna(-999,inplace=True)
    for f in X_test.columns:
        if X_test[f].dtype=='object':
            X_test[f] = X_test[f].map(lambda x:x if x in set(X_train_123[f]) else -999)
    for f in X_test.columns:
        if X_test[f].dtype=='object': 
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(X_train_123[f])+[-999])
            X_test[f] = lbl.transform(list(X_test[f])) 
    pred_value = 0
    for model in models:
        pred_value += model.predict_proba(X_test)/5
    pred_data = list(raw_y_to_array2_test(pred_value,num_counts,bins,X_test['dist_to_end']))
    pred_data = np.array(pred_data).reshape(1,199)
    
    ### ###########################NN part
    for f in basetable.columns:
        if basetable[f].dtype=='object':
            basetable[f] = basetable[f].map(lambda x:x if x in set(train_single123[f]) else np.nan)
    cat = ['Team','NflId','Down','OffensePersonnel','DefensePersonnel','Position','GameWeather']
    for column in basetable.columns:
        if column not in cat:
            if pd.isna(train_single123[column].median()):
                basetable[column] = basetable[column].fillna(-1)
            else:
                basetable[column] = basetable[column].fillna(train_single123[column].median())
        else:
            basetable[column] = basetable[column].fillna(mode(train_single123[column])[0][0])
    for f in basetable.columns:
        if basetable[f].dtype=='object': 
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(X_train_123_nn[f]))
            basetable[f] = lbl.transform(list(basetable[f])) 

    scaled_basetable = scaler.transform(basetable)
    y_pred = predict(scaled_basetable)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
    y_pred[0,(basetable['dist_to_end'].values[0]+99):] = 1
    y_pred = y_pred.reshape(1,199)
    
    #### ensemble
    
    pred_data1 = 0.75*pred_data + 0.25*y_pred
    
    
    
    ############alll
    
    pred_target = pd.DataFrame(index = sample_prediction_df.index, \
                               columns = sample_prediction_df.columns, \
                               #data = np.array(pred_data))
                               data = pred_data1)
    #print(pred_target)
    env.predict(pred_target)
env.write_submission_file()
    

/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.6/site-packages/scipy/stats/stats.py:248: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)
/opt/conda/lib/python3.6/site-packages/scipy/stats/stats.py:248: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)
/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scala

Your submission file has been saved!  Once you `Commit` your Notebook and it finishes running, you can submit the file to the competition from the Notebook Viewer `Output` tab.


In [61]:
time_end=time.time()
print('totally cost',time_end-time_start)

totally cost 6394.22597694397
